# Title

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * a
  

## Script style function call <a id="Script_style_function_call"></a>

SoS allows you to write Python functions that accept a script (string) as the first parameter in a special script format. For example, an SoS function `R` that executes a script

```sos
R("""
pdf('output.pdf')
plot(0, 0)
dev.off()
""", workdir='result')
```

can be written as

```sos
R:  workdir='result'
pdf('output.pdf')
plot(0, 0)
dev.off()
```

SoS **automatically dedents scripts in script format** so that you can indent the scripts (add equal amount of leading white spaces to all lines) and write the above example as

```sos
R:  workdir='result'
   pdf('output.pdf')
   plot(0, 0)
   dev.off()
```

The latter is much preferred because it avoids trouble if the script contains SoS-like syntax such as `[1]` and `option:` and be treated as SoS directives, and more importantly, allows starting a new statement from a non-indented line. For example, `print('Hello world')` would be considered part of a R script in

```sos
R:  workdir='result'
pdf('output.pdf')
plot(0, 0)
dev.off()

print('Hello world')
```

but a separate statement in 

```sos
R:  workdir='result'
   pdf('output.pdf')
   plot(0, 0)
   dev.off()

print('Hello world')
```

The script format also accepts an option `expand` that **expands expressions enclosed in specified sigils** ([literal string interpolation](https://www.python.org/dev/peps/pep-0498/)). Using `expand=True`, the included script is converted to a Python f-string so that the string will be interpolated when it is executed. For example,

```
output = 'output.pdf'
R: workdir='result', expand=True
   pdf('{output}')
   plot(0, 0)
   dev.off()
```

will be translated to

```
output = 'output.pdf'
R(f'''\
pdf('{output}')
plot(0, 0)
dev.off()
''', workdir='result')
```
which will be expanded to

```
R('''\
pdf('output.pdf')
plot(0, 0)
dev.off()
''', workdir='result')
```
when the action is executed.

Although the use of expressions in literal string (`{ }` parts) is very convenient, the braces might conflict with the braces in languages such as R for which braces are widely used. The official method to include braces in such literal strings is to double the braces (`{{ }}`)  as in the following example

In [50]:
a = 1
R: expand=True
    if ({a} == 1) {{
      cat("This is 1\n");
    }} else {{
      cat("This is not 1\n");
    }}

This is 1


However, as you can imagine, inclusion of long scripts with lots of braces like this could be tedious and error-prone. For this reason, SoS allows you to use an alternative sigil for the format string. For example, using `expand='${ }'`, you can expand expressions inside `${ }` instead of `{ }` for string interpolation and do not have to double the braces.

In [51]:
a = 1
R: expand='${ }'
    if (${a} == 1) {
      cat("This is 1\n");
    } else {
      cat("This is not 1\n");
    }

This is 1


The sigil specified by parameter `expand` should be a string with a single space separating left and right sigils (e.g. `"[ ]"`, `"s( )"`, `"< >"`). Note that the use of alternative sigils is just a convenient way for you to include large trunks of scripts with braces. Under the hood SoS will convert the string to properly formatted f-strings with doubled braces and `{ }` sigils.

As a final note, the SoS script format could be confused with Python's type hint syntax but SoS is able to tell the differences and treat them correctly:

In [52]:
var : str = 5
var

5

### Option `sigil`
 <a id="Option_sigil"></a>
Option `sigil` accepts a string for an alternative sigil, or `None` to disable string interpolation in the step. The sigil must be two strings separated by a space, such as `%( )`, `< >`, and `#{ }`. Sigils with equal left and right symbol such as `# #` can be used although they do not support features such as nested interpolation. Please refer to section [SoS Syntax](SoS_Syntax.html) for details of this option.

## Further reading

* 